In [ ]:
import os
import andes
import numpy as np
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import scipy.stats as stat
import seaborn as sns

from andes.interop.pandapower import to_pandapower, runopp_map
from andes.interop.pandapower import add_gencost, build_group_table
andes.config_logger(stream_level=20)
from math import ceil, floor

# virtual inertia scheduling class
from pdpower import make_link_table
from opf import dcopf
from vistds import get_andes_case

In [ ]:
# ini andes attribute
case_path = '/VIS_opf/ieee39_vis_tds.xlsx'
ssa = get_andes_case(case_path)
ssa.TDS.config.no_tqdm = 1 # trun off tds progress fig to speed up sim

# ini load
ptotal = ssa.PQ.as_df()['p0'].sum()
gentotal = ssa.PV.as_df()['Sn'].sum() + ssa.Slack.as_df()['Sn'].sum()
gentotal /= ssa.config.mva
l_rate = ptotal/gentotal

print(f'Gen capacity is: {gentotal} p.u.\
        \nInitial load is: {ptotal.round(4)} p.u.\
        \ninitial load rate is {l_rate*100} %' )


In [ ]:
ss = dcopf(ssa)

# add gen cost:
# Note: if two gens have the same cost, optimization results of
#       pandapower and gurobipy may be different.
linearcost = [
                1,   # PV_1 vsg
                20, 
                20,
                20, 
                20, 
                1,   # PV_6 vsg
                20, 
                1,   # PV_8 vsg
                1,   # PV_9 vsg 
                20   # slack
            ]

quadraticcost = [
                0.001,   # PV_1 vsg
                0.014,
                0.020,
                0.019,
                0.021,
                0.001,   # PV_6 vsg
                0.020,
                0.001,   # PV_8 vsg
                0.001,   # PV_9 vsg
                0.026    # slack
            ]
for i in range(10):
    linearcost[i] /= 12
    quadraticcost[i] /= 12
    
# set up cost
ss.cost['c1'] = linearcost
ss.cost['c2'] = quadraticcost
ss.update_dict()